## 1. API키 가져오기

In [1]:
from dotenv import load_dotenv
load_dotenv("openai.env")

#import os
#print(os.getenv("OPENAI_API_KEY"))

True

## 2. 목차 추출
#### 메타데이터로 활용하기 위함

In [2]:
import pdfplumber
import re

pdf_path = "./PromptEngineering.pdf"
toc_pages = [2, 3, 4]  # 페이지 3, 4, 5

main_headings = []
sub_headings = []
sub_sub_headings = []

with pdfplumber.open(pdf_path) as pdf:
    for page_num in toc_pages:
        page = pdf.pages[page_num]
        words = page.extract_words(use_text_flow=True)

        lines = {}
        for word in words:
            top = round(word['top'])
            x0 = word['x0']
            text = word['text']
            if top not in lines:
                lines[top] = []
            lines[top].append((x0, text))

        for top in sorted(lines):
            line = lines[top]
            line.sort()  # 왼쪽에서 오른쪽 정렬
            x0_first = line[0][0]
            text_line = " ".join(word for _, word in line)
            text_line = re.sub(r'\s+\d+$', '', text_line).strip()

            # 위치(x0 값) 기반으로 heading level 결정
            if x0_first <= 75:
                main_headings.append(text_line)
            elif x0_first <= 100:
                sub_headings.append(text_line)
            else:
                sub_sub_headings.append(text_line)

# 출력 확인
print(f"📘 Main Headings ({len(main_headings)}):\n{main_headings}\n")
print(f"📗 Sub Headings ({len(sub_headings)}):\n{sub_headings}\n")
print(f"📙 Sub-Sub Headings ({len(sub_sub_headings)}):\n{sub_sub_headings}\n")

📘 Main Headings (8):
['Table of contents', 'Introduction', 'Prompt engineering', 'LLM output configuration', 'Prompting techniques', 'Best Practices', 'Summary', 'Endnotes']

📗 Sub Headings (25):
['Output length', 'Sampling controls', 'General prompting / zero shot', 'One-shot & few-shot', 'System, contextual and role prompting', 'Step-back prompting', 'Chain of Thought (CoT)', 'Self-consistency', 'Tree of Thoughts (ToT)', 'ReAct (reason & act)', 'Automatic Prompt Engineering', 'Code prompting', 'Provide examples', 'Design with simplicity', 'Be specific about the output', 'Use Instructions over Constraints', 'Control the max token length', 'Use variables in prompts', 'Experiment with input formats and writing styles', 'For few-shot prompting with classification tasks, mix up the classes', 'Adapt to model updates', 'Experiment with output formats', 'Experiment together with other prompt engineers', 'CoT Best practices', 'Document the various prompt attempts']

📙 Sub-Sub Headings (11):
[

## 3. PDF 문서 읽어오기

In [3]:
from langchain.schema import Document
import pdfplumber

documents = []

# 페이지의 header/footer 제거용 crop 좌표 (상단 10%, 하단 10% 제거)
crop_coords = [0, 0.1, 1, 0.9]

# (텍스트 줄, 해당 페이지 번호)를 담는 리스트
lines_with_page = []

with pdfplumber.open(pdf_path) as pdf:
    for page_number, page in enumerate(pdf.pages, start=1):
        cropped_width = page.width
        cropped_height = page.height

        cropped_bbox = (
            crop_coords[0] * cropped_width,
            crop_coords[1] * cropped_height,
            crop_coords[2] * cropped_width,
            crop_coords[3] * cropped_height,
        )
        page_crop = page.crop(bbox=cropped_bbox)

        page_text = page_crop.extract_text()
        if page_text:
            lines = [line.strip() for line in page_text.split('\n') if line.strip()]
            for line in lines:
                lines_with_page.append((line, page_number))

# 문서 분리 기준용 변수 초기화
current_main = None
current_sub = None
current_sub_sub = None
current_content = ""
current_page = None

for line, page in lines_with_page:
    if line in main_headings:
        if current_content:
            documents.append(Document(
                page_content=current_content.strip(),
                metadata={
                    "title": pdf_path,
                    "page": current_page,
                    "heading": current_main,
                    "subheading": current_sub or "None",
                    "sub_subheading": current_sub_sub or "None"
                }
            ))
            current_content = ""
            current_sub = None
            current_sub_sub = None
        current_main = line
        current_page = page

    elif line in sub_headings:
        if current_content:
            documents.append(Document(
                page_content=current_content.strip(),
                metadata={
                    "title": pdf_path,
                    "page": current_page,
                    "heading": current_main,
                    "subheading": current_sub or "None",
                    "sub_subheading": current_sub_sub or "None"
                }
            ))
            current_content = ""
            current_sub_sub = None
        current_sub = line
        current_page = page

    elif line in sub_sub_headings:
        if current_content:
            documents.append(Document(
                page_content=current_content.strip(),
                metadata={
                    "title": pdf_path,
                    "page": current_page,
                    "heading": current_main,
                    "subheading": current_sub or "None",
                    "sub_subheading": current_sub_sub or "None"
                }
            ))
            current_content = ""
        current_sub_sub = line
        current_page = page

    else:
        current_content += line + " "
        current_page = page  # 본문일 경우에도 현재 페이지로 업데이트

# 마지막 청크 추가
# 미실행시 마지막 청크가 누락되거나 빈 청크가 생길 수 있음
if current_content:
    documents.append(Document(
        page_content=current_content.strip(),
        metadata={
            "title": pdf_path,
            "page": current_page,
            "heading": current_main,
            "subheading": current_sub or "None",
            "sub_subheading": current_sub_sub or "None"
        }
    ))

## 4. Chunk 나누기

### 옵션1. 목차대로 나누기
#### 장점 : 문서 작성자 의도대로 나눌 수 있음
#### 단점 : 각 청크의 글자수 불균형 (최소 192자, 최대 6311자)

In [4]:
'''
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(separator="\n\n")
docs = splitter.split_documents(documents)

print(f"총 chunk 수: {len(docs)}")
# 각 청크의 글자 수 확인
for i, doc in enumerate(docs, 1):
    print(f"청크 {i}: {len(doc.page_content)}자")
'''

'\nfrom langchain.text_splitter import CharacterTextSplitter\n\nsplitter = CharacterTextSplitter(separator="\n\n")\ndocs = splitter.split_documents(documents)\n\nprint(f"총 chunk 수: {len(docs)}")\n# 각 청크의 글자 수 확인\nfor i, doc in enumerate(docs, 1):\n    print(f"청크 {i}: {len(doc.page_content)}자")\n'

### 옵션2. 500자 기준으로 나누기
#### 장점 : 각 청크의 글자수 비교적 균형적, 옵션1에서 500자 이하인 청크는 그대로 유지됨
#### 단점 : 연결된 내용이 서로 다른 청크들에 포함될 수 있음

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)

print(f"총 chunk 수: {len(docs)}")
# 각 청크의 글자 수 확인
for i, doc in enumerate(docs, 1):
    print(f"청크 {i}: {len(doc.page_content)}자")

총 chunk 수: 186
청크 1: 291자
청크 2: 497자
청크 3: 498자
청크 4: 389자
청크 5: 488자
청크 6: 497자
청크 7: 410자
청크 8: 499자
청크 9: 497자
청크 10: 487자
청크 11: 370자
청크 12: 252자
청크 13: 494자
청크 14: 296자
청크 15: 439자
청크 16: 491자
청크 17: 494자
청크 18: 351자
청크 19: 493자
청크 20: 498자
청크 21: 285자
청크 22: 494자
청크 23: 494자
청크 24: 497자
청크 25: 492자
청크 26: 498자
청크 27: 416자
청크 28: 498자
청크 29: 88자
청크 30: 494자
청크 31: 496자
청크 32: 490자
청크 33: 496자
청크 34: 153자
청크 35: 498자
청크 36: 496자
청크 37: 499자
청크 38: 487자
청크 39: 494자
청크 40: 418자
청크 41: 499자
청크 42: 498자
청크 43: 497자
청크 44: 305자
청크 45: 496자
청크 46: 495자
청크 47: 497자
청크 48: 493자
청크 49: 499자
청크 50: 205자
청크 51: 499자
청크 52: 496자
청크 53: 497자
청크 54: 493자
청크 55: 496자
청크 56: 496자
청크 57: 498자
청크 58: 68자
청크 59: 496자
청크 60: 496자
청크 61: 499자
청크 62: 80자
청크 63: 499자
청크 64: 499자
청크 65: 499자
청크 66: 498자
청크 67: 492자
청크 68: 492자
청크 69: 491자
청크 70: 493자
청크 71: 492자
청크 72: 495자
청크 73: 498자
청크 74: 497자
청크 75: 321자
청크 76: 496자
청크 77: 496자
청크 78: 499자
청크 79: 498자
청크 80: 499자
청크 81: 494자
청크 82: 496자
청크 83: 498자
청

In [6]:
# heading, subheading, sub_subheading 확인
print(docs[17].metadata)

{'title': './PromptEngineering.pdf', 'page': 10, 'heading': 'LLM output configuration', 'subheading': 'Sampling controls', 'sub_subheading': 'Temperature'}


In [7]:
print(docs[17])

page_content='temperature with high certainty. A higher Gemini temperature setting is like a high softmax temperature, making a wider range of temperatures around the selected setting more acceptable. This increased uncertainty accommodates scenarios where a rigid, precise temperature may not be essential like for example when experimenting with creative outputs.' metadata={'title': './PromptEngineering.pdf', 'page': 10, 'heading': 'LLM output configuration', 'subheading': 'Sampling controls', 'sub_subheading': 'Temperature'}


## 5. Qdrant 벡터DB 생성 또는 사용

In [8]:
from langchain.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings
from qdrant_client import QdrantClient
from qdrant_client.http.models import CollectionStatus

# 임베딩 모델 설정
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 로컬 도커로 실행 중인 Qdrant에 연결
client = QdrantClient(url="http://localhost:6333")

# 현재 존재하는 컬렉션 목록 확인
existing_collections = [col.name for col in client.get_collections().collections]
#print(existing_collections)

# 사용할 컬렉션 이름
collection_name = "qdrant_0526"

if collection_name in existing_collections:
    print(f"기존 컬렉션 '{collection_name}'에 연결합니다.")
    qdrant = Qdrant(
        client=client,
        collection_name=collection_name,
        embeddings=embeddings,
    )
else:
    print(f"컬렉션 '{collection_name}'을 생성하고 문서를 업로드합니다.")
    qdrant = Qdrant.from_documents(
        documents=docs,
        embedding=embeddings,
        url="http://localhost:6333",
        prefer_grpc=True,  # gRPC 사용 가능하면 True 추천
        collection_name=collection_name,
    )

기존 컬렉션 'qdrant_0526'에 연결합니다.


C:\Users\wiseai\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 0.3.0. An updated version of the class exists in the langchain-qdrant package and should be used instead. To use it run `pip install -U langchain-qdrant` and import as `from langchain_qdrant import Qdrant`.
  warn_deprecated(


## 6. 쿼리에 대한 답변 생성

#### Qdrant에서 유사 문서 검색

In [9]:
query = "CoT와 ToT를 비교해서 설명해줘"
retrieved_docs = qdrant.similarity_search_with_score(query, k=5)

for i, (doc, score) in enumerate(retrieved_docs, 1):
    print(f"\n------- 문서 {i} -------")
    print(f"유사도 점수: {score:.4f}")
    print(f"대목차: {doc.metadata.get('heading')}")
    print(f"중목차: {doc.metadata.get('subheading')}")
    print(f"소목차: {doc.metadata.get('sub_subheading')}")
    print(f"페이지: {doc.metadata.get('page')}")
    print(f"내용: {doc.page_content[:1000]}")


------- 문서 1 -------
유사도 점수: 0.4728
대목차: Prompting techniques
중목차: Tree of Thoughts (ToT)
소목차: None
페이지: 37
내용: Now that we are familiar with chain of thought and self-consistency prompting, let’s review Tree of Thoughts (ToT).12 It generalizes the concept of CoT prompting because it allows LLMs to explore multiple different reasoning paths simultaneously, rather than just following a single linear chain of thought. This is depicted in Figure 1. Figure 1. A visualization of chain of thought prompting on the left versus. Tree of Thoughts prompting on the right This approach makes ToT particularly

------- 문서 2 -------
유사도 점수: 0.4451
대목차: Prompting techniques
중목차: Chain of Thought (CoT)
소목차: None
페이지: 32
내용: Chain of Thought (CoT) 9 prompting is a technique for improving the reasoning capabilities of LLMs by generating intermediate reasoning steps. This helps the LLM generate more accurate answers. You can combine it with few-shot prompting to get better results on more complex tasks th

In [10]:
# 유사도 점수 0.4 이상으로 필터링 및 유사도 점수 제거
filtered_docs = [doc for doc, score in retrieved_docs if score >= 0.4]
if not filtered_docs:
    print("검색에 실패했습니다.")
else:
    print(f"{len(filtered_docs)}개 문서 검색에 성공했습니다.")

5개 문서 검색에 성공했습니다.


#### 답변 생성

In [11]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.chains.question_answering import load_qa_chain
from langchain.docstore.document import Document

# LLM 초기화
llm = ChatOpenAI(temperature=0, model_name="gpt-4.1")

# QA 체인 불러오기 (chain_type: "stuff", "map_reduce", "refine" 중 선택)
qa_chain = load_qa_chain(llm, chain_type="stuff")

# 질문에 대한 답변 생성
answer = qa_chain.invoke({"input_documents": filtered_docs, "question": query})

print("❓ 질문:", answer.get("question", ""))
print("\n" + "="*100 + "\n")
print("💡 답변:", answer.get("output_text", ""))

❓ 질문: CoT와 ToT를 비교해서 설명해줘


💡 답변: 네, Chain of Thought(CoT)와 Tree of Thoughts(ToT)를 비교해서 설명드리겠습니다.

### 1. Chain of Thought (CoT)
- **개념**: CoT는 대형 언어 모델(LLM)이 문제를 해결할 때, 중간 추론 과정을 단계별로 생성하도록 유도하는 프롬프트 기법입니다.
- **방식**: 한 번에 한 경로(선형적, 일직선)의 추론만을 따라가며, 각 단계별로 논리적 이유를 차례로 나열합니다.
- **장점**: 구현이 간단하고, 기존 LLM에 바로 적용할 수 있으며, 복잡한 문제에 대해 더 정확한 답변을 얻을 수 있습니다.
- **적용 예시**: 수학 문제 풀이, 논리적 추론 문제 등에서 "생각의 흐름"을 단계별로 보여줍니다.

### 2. Tree of Thoughts (ToT)
- **개념**: ToT는 CoT를 일반화한 기법으로, LLM이 여러 개의 추론 경로(가지)를 동시에 탐색할 수 있도록 합니다.
- **방식**: 문제를 해결하는 과정에서 하나의 선형 경로가 아니라, 여러 가지 가능한 중간 단계(노드)를 트리 구조로 확장하며 탐색합니다. 각 노드는 하나의 "생각"을 나타내고, 여러 노드에서 새로운 가지로 분기할 수 있습니다.
- **장점**: 복잡하고 탐색이 필요한 문제(예: 창의적 문제 해결, 계획 수립 등)에 더 적합합니다. 다양한 경로를 동시에 고려하므로 더 나은 해답을 찾을 확률이 높아집니다.
- **적용 예시**: 여러 가지 해결책을 비교하거나, 다양한 시나리오를 동시에 고려해야 하는 문제에 효과적입니다.

---

### 요약 비교

| 구분         | Chain of Thought (CoT)         | Tree of Thoughts (ToT)           |
|--------------|-------------------------------|-----------------------------------|
| 추론 경로 